In [1]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
import io
import os
import gzip
import sqlite3
from langchain.chat_models import ChatOpenAI

## **Data Engineering**
Data engineering is a critical process in the loading and preparing data for analysis by ensuring its accessibility, quality, and structure.
In the modern day and age - this is especially pertinent when it comes to integrating with databases on cloud-based platforms - such as AWS!!  

In this section, we focus on how boto3, the AWS Software Development Kit (SDK) for Python, facilitates integration with Amazon S3 to access, download, and process data. By leveraging AWS services, we can efficiently manage large datasets, even those stored in compressed formats like `.db.gz`!

In [2]:
# Create an anonymous S3 client (DISABLE SIGNATURES) --> ONLY for public datasets
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Choice of (Public) S3 bucket (amend as you choose to!)
bucket_name = "megascenes"  

In [3]:
s3_response = s3.list_objects_v2(Bucket=bucket_name, MaxKeys = 20)  # Fetch first 20 files

# Print the available file keys --> files which we can choose to access
if "Contents" in s3_response:
    print("Files in the bucket:")
    for obj in s3_response["Contents"]:
        print("-", obj["Key"])
else:
    print("No files found or access denied.")

Files in the bucket:
- databases/
- databases/descriptors/
- databases/descriptors/000/000/descriptors.db.gz
- databases/descriptors/000/001/descriptors.db.gz
- databases/descriptors/000/002/descriptors.db.gz
- databases/descriptors/000/003/descriptors.db.gz
- databases/descriptors/000/004/descriptors.db.gz
- databases/descriptors/000/005/descriptors.db.gz
- databases/descriptors/000/006/descriptors.db.gz
- databases/descriptors/000/007/descriptors.db.gz
- databases/descriptors/000/008/descriptors.db.gz
- databases/descriptors/000/009/descriptors.db.gz
- databases/descriptors/000/010/descriptors.db.gz
- databases/descriptors/000/011/descriptors.db.gz
- databases/descriptors/000/012/descriptors.db.gz
- databases/descriptors/000/013/descriptors.db.gz
- databases/descriptors/000/014/descriptors.db.gz
- databases/descriptors/000/015/descriptors.db.gz
- databases/descriptors/000/016/descriptors.db.gz
- databases/descriptors/000/017/descriptors.db.gz


In [4]:
db_bucket = list(map(lambda x:x["Key"],s3.list_objects_v2(Bucket=bucket_name)["Contents"])) 
# Similar as above - only this time no max limit
print(len(db_bucket))
db_bucket[:20]

1000


['databases/',
 'databases/descriptors/',
 'databases/descriptors/000/000/descriptors.db.gz',
 'databases/descriptors/000/001/descriptors.db.gz',
 'databases/descriptors/000/002/descriptors.db.gz',
 'databases/descriptors/000/003/descriptors.db.gz',
 'databases/descriptors/000/004/descriptors.db.gz',
 'databases/descriptors/000/005/descriptors.db.gz',
 'databases/descriptors/000/006/descriptors.db.gz',
 'databases/descriptors/000/007/descriptors.db.gz',
 'databases/descriptors/000/008/descriptors.db.gz',
 'databases/descriptors/000/009/descriptors.db.gz',
 'databases/descriptors/000/010/descriptors.db.gz',
 'databases/descriptors/000/011/descriptors.db.gz',
 'databases/descriptors/000/012/descriptors.db.gz',
 'databases/descriptors/000/013/descriptors.db.gz',
 'databases/descriptors/000/014/descriptors.db.gz',
 'databases/descriptors/000/015/descriptors.db.gz',
 'databases/descriptors/000/016/descriptors.db.gz',
 'databases/descriptors/000/017/descriptors.db.gz']

In [5]:
file_key = 'databases/descriptors/000/000/descriptors.db.gz' # Amend as you wish!
# Read the file from S3 into memory
obj_response = s3.get_object(Bucket=bucket_name, Key=file_key)
compressed_data = obj_response["Body"].read()

# Decompress the data
with gzip.GzipFile(fileobj=io.BytesIO(compressed_data)) as f:
    decompressed_data = f.read()

# Write to an in-memory database
db_buffer = io.BytesIO(decompressed_data)

# Connect SQLite to the in-memory buffer - no files being downloaded yet
conn = sqlite3.connect(":memory:",)

In [6]:
# Load the decompressed database into .db file for SQLITE to connect to
with open("temp.db", "wb") as temp_db:
    temp_db.write(db_buffer.getvalue())

# Reconnect to the temporary SQLite database
conn = sqlite3.connect("temp.db")

In [7]:
table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'",conn) # Get table names of those availabe in the database
table_names

,name
0,descriptors


In [8]:
# Read the table into pandas
df = pd.read_sql_query("SELECT * FROM descriptors", conn)
conn.close()

## **GenAI & Data Extraction**

The code listed above is a pretty standard way to extract data from an AWS bucket (one that is publicly available) to one's local machine. Yet coding it out time and time again for various datasets from a number of buckets is bound to be time consuming and not supremely productive. 

Indeed - it would be thus preferential to have someone/ something else handle this for us as Data Engineers - and that's where AI comes in! However, the task is not truly as simple as passing in the url of interest and asking 'Chat - please read this for me, thank you' (as much as one may wish that is the case). 

Thus - the data engineer must use his/her prior knowledge to instruct the model clearly on what to do to produce reliable outputs as need be - and also be prepared to troubleshoot it wherever necessary! 

In [9]:
with open("SDS_OpenAI_key.txt", "r") as f:
    api_key = f.read().strip()
    os.environ["OPENAI_API_KEY"] = api_key

llm = ChatOpenAI(model_name="gpt-4", temperature=0.001) # the temperature setting can be thought of as a way to set the variability of the 
# result generated - as reliable code is expected as an output - the number passed into it is fairly low to get *somewhat* consistent
# outputs of code after a number of runs

C:\Users\Acer\AppData\Local\Temp\ipykernel_1520\2459014232.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0.001) # the temperature setting can be thought of as a way to set the variability of the


In [10]:
## Ideal prompt
prompt = f"""Chat, given that I am using a publicly available s3 database
with bucket {bucket_name} and file {file_key} being parsed through boto3 
via an s3 client with UNSIGNED credentials from botocore in Python. 
How can I read the specific file type of {file_key} into a pandas based dataframe? 
I only want my output to be in code form like ```df = pd.read_sql_query()``` and not have any human text present. 
The file may be temporarily stored locally should its size be less than 500 MB
"""

In [11]:
chat_response = llm.invoke(prompt)
print(chat_response.content)

```python
import pandas as pd
import boto3
import botocore
from sqlalchemy import create_engine
import gzip
import shutil
import os

# Create an UNSIGNED S3 client
s3 = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

bucket_name = 'megascenes'
file_key = 'databases/descriptors/000/000/descriptors.db.gz'
local_file_name = 'descriptors.db.gz'
local_unzipped_file_name = 'descriptors.db'

# Download the file from S3
s3.download_file(bucket_name, file_key, local_file_name)

# Unzip the file
with gzip.open(local_file_name, 'rb') as f_in:
    with open(local_unzipped_file_name, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Create a database connection
engine = create_engine('sqlite:///' + local_unzipped_file_name)

# Read the SQL database file into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM your_table_name", engine)

# Remove the local files
os.remove(local_file_name)
os.remove(local_unzipped_file_name)
```
Please replace `"SE

In [12]:
### Output from ChatGPT!

import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import gzip
import sqlite3
from urllib.parse import urlparse

# Parse the S3 URL
url = urlparse('s3://megascenes/databases/descriptors/000/000/descriptors.db.gz')

# Create an S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

# Download the file
with open('descriptors.db.gz', 'wb') as f:
    s3.download_fileobj(url.netloc, url.path.lstrip('/'), f)

# Unzip the file
with gzip.open('descriptors.db.gz', 'rb') as f_in:
    with open('descriptors.db', 'wb') as f_out:
        f_out.write(f_in.read())

# Connect to the SQLite database
conn = sqlite3.connect('descriptors.db')

# Read the SQL query into a DataFrame
df = pd.read_sql_query("SELECT * FROM descriptors", conn)

# Close the connection
conn.close()

In [13]:
# Removal of files from memory
os.remove("temp.db")
os.remove("descriptors.db")
os.remove("descriptors.db.gz")

## **GenAI & Transformation**

As we have explored previously, GenAI can be a very useful tool for a data engineer to craft applicable code for the parsing of databases from cloud base sources - especially when a good prompt is used. But that is not the most value adding stage for GenAI on its own in the ETL pipeline/ in pipelining in General. 

Rather - GenAI can be integrated in the Transformation stage on our data, as well as on the Analytical stage - to offer insights to ways in which our data can be transformed (ideally beyond the standard ways of imputing missing values/ changing datatypes) for our benefit!

In [14]:
df_prev = pd.read_csv(r"data\yellow_tripdata_2019-01.csv")
df_recent = pd.read_csv(r"data\yellow_tripdata_2020-01.csv")

C:\Users\Acer\AppData\Local\Temp\ipykernel_1520\249021592.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_recent = pd.read_csv(r"data\yellow_tripdata_2020-01.csv")


In [15]:
df_recent.sample(100) # sample of the `recent` dataframe 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
6377519,NaN,2020-01-15 17:37:45,2020-01-15 18:09:41,NaN,5.94,NaN,NaN,231,225,NaN,26.16,2.75,0.0,0.00,0.00,0.3,29.21,0.0
924522,1.0,2020-01-06 12:54:11,2020-01-06 13:21:07,1.0,19.70,2.0,N,132,262,1.0,52.00,2.50,0.5,12.25,6.12,0.3,73.67,2.5
122714,1.0,2020-01-01 16:24:30,2020-01-01 16:57:45,1.0,18.30,2.0,N,230,132,2.0,52.00,2.50,0.5,0.00,6.12,0.3,61.42,2.5
2564446,2.0,2020-01-14 11:46:45,2020-01-14 12:06:04,1.0,2.98,1.0,N,100,231,1.0,13.50,0.00,0.5,3.36,0.00,0.3,20.16,2.5
2974733,1.0,2020-01-16 09:23:12,2020-01-16 09:32:34,0.0,1.10,1.0,N,74,41,1.0,7.50,0.00,0.5,0.70,0.00,0.3,9.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530376,2.0,2020-01-18 17:56:37,2020-01-18 18:04:57,1.0,1.19,1.0,N,144,4,1.0,7.00,0.00,0.5,2.06,0.00,0.3,12.36,2.5
4667487,2.0,2020-01-24 11:40:57,2020-01-24 12:01:22,1.0,2.35,1.0,N,186,143,2.0,14.00,0.00,0.5,0.00,0.00,0.3,17.30,2.5
4804576,2.0,2020-01-24 22:37:12,2020-01-24 22:47:01,1.0,1.28,1.0,N,90,114,1.0,8.00,0.50,0.5,1.25,0.00,0.3,13.05,2.5
1631371,2.0,2020-01-09 20:26:13,2020-01-09 20:31:34,1.0,0.73,1.0,N,211,114,1.0,5.50,0.50,0.5,1.86,0.00,0.3,11.16,2.5


In [16]:
nrows = 20

## Ideal prompt
prompt = f"""Chat, given that I have 2 dataframes {df_prev.sample(nrows)} collected in 2019 and
{df_recent.sample(nrows)} collected in 2020 - identify the relationships between these dataframes - their differences
and also possible transformations that can be applied to the relevant columns in python based code. 
"""

In [17]:
chat_response = llm.invoke(prompt)
print(chat_response.content)

The two dataframes seem to be taxi trip data collected in the years 2019 and 2020 respectively. They have the same structure and columns, which include details about the trip such as pickup and dropoff times, locations, distance, fare, and payment details.

Differences:
1. The data is collected in different years (2019 and 2020).
2. The actual data in the rows will be different as they represent different taxi trips.
3. There might be differences in the data quality or completeness between the two years. For example, the 'congestion_surcharge' column in the 2019 dataframe has some missing values (NaN), while it seems to be complete in the 2020 dataframe.

Possible Transformations:
1. Date-Time Conversion: The 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns are currently in string format. They can be converted to datetime format for further analysis. This can be done using pandas to_datetime function.
   ```
   df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datet

In [18]:
prompt = f"""Chat, given the dataframe {df_recent.sample(10)} - outline useful transformations that can be applied to 
the data to make it robust to new possible entries that can be received in the future - 
do also give the relevant python code and libraries required to achieve these tasks. 
Note that the column names are {df_recent.columns}.
Ensure these are NOT simplistic transformations like standardisation/ missing value imputation.
Give me at least 15 distinct transformations - with the relevant code being present. Analyse the dataset itself to build deep 
and useful transformations that can be mapped onto it - including the creation of new columns
"""

## Ideal prompt

In [19]:
chat_response = llm.invoke(prompt)
print(chat_response.content)

1. **Datetime Conversion**: Convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' from string to datetime format. This will allow for more complex time-based analysis and feature engineering.

```python
import pandas as pd
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
```

2. **Trip Duration**: Calculate the duration of each trip in minutes. This could be a useful feature for predicting fare amounts or tip amounts.

```python
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
```

3. **Hour of Day**: Extract the hour of day from the pickup and dropoff times. This could be useful for understanding patterns related to time of day.

```python
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['dropoff_hour'] = df['tpep_dropoff_datetime'].dt.hour
```

4. **Day of Week**: Extract the day of the week from the pickup and dropoff 

In [20]:
### Output from ChatGPT!

# **Datetime Conversion**: Convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' from string to datetime format. 
# This will allow for more complex time-based analysis and feature engineering

df_recent['tpep_pickup_datetime'] = pd.to_datetime(df_recent['tpep_pickup_datetime'])
df_recent['tpep_dropoff_datetime'] = pd.to_datetime(df_recent['tpep_dropoff_datetime'])

# **Day of Week**: Extract the day of the week from the pickup datetime. This could be useful for understanding patterns in taxi usage.

df_recent['pickup_dayofweek'] = df_recent['tpep_pickup_datetime'].dt.dayofweek

# **Is Weekend**: Determine if the pickup or dropoff is on a weekend.

df_recent['pickup_is_weekend'] = df_recent['pickup_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)

# **Hour of Day**: Extract the hour of the day from the pickup datetime. This could be useful for understanding patterns in taxi usage.

df_recent['pickup_hour_of_day'] = df_recent['tpep_pickup_datetime'].dt.hour

# **Rush Hour**: Determine if the pickup or dropoff is during rush hour (typically 7-9 AM and 4-6 PM).

df_recent['pickup_rush_hour'] = df_recent['pickup_hour_of_day'].apply(lambda x: 1 if 7 <= x <= 9 or 16 <= x <= 18 else 0)